# 模型构建

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torch.utils.data as data
from torch.utils.data import DataLoader



## 定义模型A

In [2]:
# 定义模型A
class ModelA(nn.Module):
    def __init__(self):
        super(ModelA, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv3d(64, 3, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



## 定义模型B

In [3]:
# 定义模型B
class ModelB(nn.Module):
    def __init__(self):
        super(ModelB, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(7, 64, kernel_size=3, padding=1),  # 7 channels: 3 for velocity, 4 for abcd
            nn.ReLU(),
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv3d(64, 3, kernel_size=3, padding=1)
        )

    def forward(self, x, abcd):
        x = torch.cat([x, abcd], dim=1)  # Concatenate velocity and abcd inputs
        x = self.encoder(x)
        x = self.decoder(x)
        return x

##  示例训练循环（适用于模型A和模型B）

#### 模型A的训练

In [4]:

def train(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(dataloader.dataset)

#### 模型B的训练

In [5]:
# 模型B的训练函数需要接受三项输入
def train_model_b(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for masked_velocity, abcd, targets in dataloader:
        masked_velocity, abcd, targets = masked_velocity.to(device), abcd.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(masked_velocity, abcd)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * masked_velocity.size(0)
    return running_loss / len(dataloader.dataset)

## 定义CUDA

In [6]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

##  实例化模型

In [7]:
model_a = ModelA().to(device)
model_b = ModelB().to(device)

## 定义损失和优化器

In [8]:
criterion = nn.MSELoss()
optimizer_a = optim.Adam(model_a.parameters(), lr=0.001)
optimizer_b = optim.Adam(model_b.parameters(), lr=0.001)

## 定义数据类

In [9]:
from torch.utils.data import Dataset, DataLoader

In [10]:
class VelocityDataset(Dataset):
    def __init__(self, velocity_files, abcd_files=None, mask_ratio=0.2):
        """
        velocity_files: list of paths to velocity npy files
        abcd_files: list of paths to abcd npy files (only for Model B)
        mask_ratio: percentage of the velocity field to mask (e.g., 0.2 means 20% of the field will be masked)
        """
        self.velocity_files = velocity_files
        self.abcd_files = abcd_files
        self.mask_ratio = mask_ratio

    def __len__(self):
        return len(self.velocity_files)

    def __getitem__(self, idx):
        # 加载速度场
        velocity = np.load(self.velocity_files[idx])

        # 生成掩码
        mask = np.random.rand(*velocity.shape) < self.mask_ratio
        masked_velocity = velocity.copy()
        masked_velocity[mask] = 0  # 掩盖部分速度场数据

        # 转换为PyTorch张量
        masked_velocity = torch.from_numpy(masked_velocity).float()
        velocity = torch.from_numpy(velocity).float()

        if self.abcd_files is not None:
            # 加载并转换物理量 a, b, c, d
            abcd = np.load(self.abcd_files[idx])
            abcd = torch.from_numpy(abcd).float()
            return masked_velocity, abcd, velocity  # 返回模型B的输入和目标
        else:
            return masked_velocity, velocity  # 返回模型A的输入和目标


### 获取训练和测试集的文件列表

In [11]:
import os

In [12]:
train_velocity_files = [os.path.join('data/processed/train/velocities', f) for f in os.listdir('data/processed/train/velocities')]
train_abcd_files = [os.path.join('data/processed/train/abcd', f) for f in os.listdir('data/processed/train/abcd')]

test_velocity_files = [os.path.join('data/processed/test/velocities', f) for f in os.listdir('data/processed/test/velocities')]
test_abcd_files = [os.path.join('data/processed/test/abcd', f) for f in os.listdir('data/processed/test/abcd')]

### 创建Dataset实例

In [13]:
from datasets import NpyDataset

In [14]:
train_dataset_a = NpyDataset(velocity_dir='data/processed/train/velocities', mask_ratio=0.2)
train_dataset_b = NpyDataset(velocity_dir='data/processed/train/velocities', abcd_dir='data/processed/train/abcd', mask_ratio=0.2)


In [15]:
test_dataset_a = NpyDataset(velocity_dir='data/processed/test/velocities', mask_ratio=0.2)
test_dataset_b = NpyDataset(velocity_dir='data/processed/test/velocities', abcd_dir='data/processed/test/abcd', mask_ratio=0.2)


### 创建DataLoader实例

In [16]:
train_loader_a = DataLoader(train_dataset_a, batch_size=4, shuffle=True, num_workers=0)
train_loader_b = DataLoader(train_dataset_b, batch_size=4, shuffle=True, num_workers=0)


In [17]:
test_loader_a = DataLoader(test_dataset_a, batch_size=4, shuffle=False, num_workers=0)
test_loader_b = DataLoader(test_dataset_b, batch_size=4, shuffle=False, num_workers=0)


# 模型训练

In [18]:
import torch.utils.data as data
from torch.utils.data import DataLoader


## 定义测试

### 测试函数

In [19]:
def validate(model, dataloader, criterion, device, is_model_b=False):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for data in dataloader:
            if is_model_b:
                masked_velocity, abcd, targets = data
                masked_velocity, abcd, targets = masked_velocity.to(device), abcd.to(device), targets.to(device)
                outputs = model(masked_velocity, abcd)
            else:
                inputs, targets = data
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
            
            loss = criterion(outputs, targets)
            running_loss += loss.item() * targets.size(0)
    
    avg_loss = running_loss / len(dataloader.dataset)
    return avg_loss


In [20]:
num_epochs = 10
for epoch in range(num_epochs):
    train_loss_a = train(model_a, train_loader_a, optimizer_a, criterion, device)
    val_loss_a = validate(model_a, test_loader_a, criterion, device, is_model_b=False)
    print(f"Epoch {epoch+1}/{num_epochs}, Model A Train Loss: {train_loss_a:.4f}, Validation Loss: {val_loss_a:.4f}")


Epoch 1/10, Model A Train Loss: 0.0003, Validation Loss: 0.0002
Epoch 2/10, Model A Train Loss: 0.0002, Validation Loss: 0.0002
Epoch 3/10, Model A Train Loss: 0.0002, Validation Loss: 0.0001
Epoch 4/10, Model A Train Loss: 0.0002, Validation Loss: 0.0001
Epoch 5/10, Model A Train Loss: 0.0002, Validation Loss: 0.0001
Epoch 6/10, Model A Train Loss: 0.0001, Validation Loss: 0.0001
Epoch 7/10, Model A Train Loss: 0.0001, Validation Loss: 0.0001
Epoch 8/10, Model A Train Loss: 0.0001, Validation Loss: 0.0001
Epoch 9/10, Model A Train Loss: 0.0001, Validation Loss: 0.0001
Epoch 10/10, Model A Train Loss: 0.0001, Validation Loss: 0.0001


In [21]:
for epoch in range(num_epochs):
    train_loss_b = train_model_b(model_b, train_loader_b, optimizer_b, criterion, device)
    val_loss_b = validate(model_b, test_loader_b, criterion, device, is_model_b=True)
    print(f"Epoch {epoch+1}/{num_epochs}, Model B Train Loss: {train_loss_b:.4f}, Validation Loss: {val_loss_b:.4f}")


Epoch 1/10, Model B Train Loss: 0.0004, Validation Loss: 0.0003
Epoch 2/10, Model B Train Loss: 0.0002, Validation Loss: 0.0002
Epoch 3/10, Model B Train Loss: 0.0002, Validation Loss: 0.0002
Epoch 4/10, Model B Train Loss: 0.0002, Validation Loss: 0.0002
Epoch 5/10, Model B Train Loss: 0.0002, Validation Loss: 0.0001
Epoch 6/10, Model B Train Loss: 0.0002, Validation Loss: 0.0001
Epoch 7/10, Model B Train Loss: 0.0002, Validation Loss: 0.0001
Epoch 8/10, Model B Train Loss: 0.0002, Validation Loss: 0.0001
Epoch 9/10, Model B Train Loss: 0.0002, Validation Loss: 0.0001
Epoch 10/10, Model B Train Loss: 0.0001, Validation Loss: 0.0001
